
- call run_pyscf for all of them via slurm

which python is used?
in which directory is python called?

sbatch shell-script

shell-script:
python path_to/run_pyscf.py


In [11]:
import sys
sys.path.insert(0,'/home/misa/git_repositories/APDFT/prototyping/atomic_energies')
import alchemy_tools_pyscf as atp
from parse_pyscf import read_input
import qml
import numpy as np
import os
import utils_qm as uqm

# get current directory

run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_003664'
#run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_001711'  #os.getcwd()
os.chdir(run_dir)
print('Initializing')
input_parameters = read_input(os.path.join(run_dir, 'input_parameters'))
inputfile = input_parameters['structure_file']#os.path.join(basepath, com+'.xyz')
intg_meth = input_parameters['intg_meth']
basis = input_parameters['basis'] # 'def2-qzvp'
com = qml.Compound(xyz=inputfile)

lam_vals = np.array([2, 27, 54])/54#np.arange(2, 54, 2)/52
#lam_vals = np.concatenate((np.zeros(1), lam_vals))
alchemical_potentials = []
#alchemical_potentials.append(np.zeros(len(com.nuclear_charges)).tolist())
mo_energies = []
mo_occs = []
dm_restart = None

Initializing


In [12]:
deltaZ, includeonly, mol = atp.prepare_input(com.coordinates, com.nuclear_charges, 0, basis)

In [13]:
mol.energy_nuc()

295.8829146919174

In [1]:
import sys
sys.path.insert(0,'/home/misa/git_repositories/APDFT/prototyping/atomic_energies')
import alchemy_tools_pyscf as atp
from parse_pyscf import read_input
import qml
import numpy as np
import os
import utils_qm as uqm

# get current directory

run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_005352'
#run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_001711'  #os.getcwd()
os.chdir(run_dir)
print('Initializing')
input_parameters = read_input(os.path.join(run_dir, 'input_parameters'))
inputfile = input_parameters['structure_file']#os.path.join(basepath, com+'.xyz')
intg_meth = input_parameters['intg_meth']
basis = input_parameters['basis'] # 'def2-qzvp'
com = qml.Compound(xyz=inputfile)

lam_vals = np.array([2, 27, 54])/54#np.arange(2, 54, 2)/52
#lam_vals = np.concatenate((np.zeros(1), lam_vals))
alchemical_potentials = []
#alchemical_potentials.append(np.zeros(len(com.nuclear_charges)).tolist())
mo_energies = []
mo_occs = []
dm_restart = None

# print('Starting calculations')
for lam in lam_vals: #np.flip(lam_vals)[:-1]:
    num_elec = atp.get_num_elec(lam, com.nuclear_charges.sum())
    print(f'Doing calculation for lambda = {lam}, number of electrons = {num_elec}')
    print('Preparing input')
    deltaZ, includeonly, mol = atp.prepare_input(com.coordinates, com.nuclear_charges, num_elec, basis)
    print('Doing SCF calculation')
    dm, e_tot, mo_energy, mo_occ = atp.make_apdft_calc(deltaZ, dm_restart, includeonly, mol, method = "HF", **{'max_cycle':300, 'init_guess':'atom'})
#     dm_restart = dm
    mo_energies.append(mo_energy)
    mo_occs.append(mo_occ)
    print('Calculating alchemical potentials')
    alchpots_lambda = atp.calculate_alchpot(dm, includeonly, mol)
    alchemical_potentials.append(alchpots_lambda) 
#   saving results
    uqm.save_obj(alchemical_potentials, os.path.join(run_dir, 'alchemical_potentials_tmp'))
    uqm.save_obj(mo_energies, os.path.join(run_dir, 'mo_energies_tmp'))
    uqm.save_obj(mo_occs, os.path.join(run_dir, 'mo_occupancies_tmp'))
    uqm.save_obj(lam_vals[:np.where(lam_vals==lam)[0][0]+1], os.path.join(run_dir, 'lam_vals_tmp'))
    
alchemical_potentials = np.array(alchemical_potentials)
# average_potentials = atp.calculate_average_alchpots(alchemical_potentials, lam_vals, intg_meth)

print('Saving results')
# save alchemical_potentials
file_raw = os.path.join(run_dir, 'alchpots')
np.save(file_raw, alchemical_potentials, allow_pickle=False)
# save mo energies
file_moen = os.path.join(run_dir, 'mo_energies')
np.save(file_moen, mo_energies, allow_pickle=False)
# save mo occupancies
file_moocc = os.path.join(run_dir, 'mo_occupancies')
np.save(file_moocc, mo_occs, allow_pickle=False)
# save lam_vals
np.savetxt(os.path.join(run_dir, 'lam_vals.txt'),lam_vals)

Initializing
Doing calculation for lambda = 0.037037037037037035, number of electrons = 2
Preparing input
Doing SCF calculation


Overwritten attributes  energy_nuc  of <class 'pyscf.qmmm.itrf.qmmm_for_scf.<locals>.QMMM'>
/home/misa/anaconda3/envs/atomic-energies/lib/python3.7/site-packages/pyscf/gto/mole.py:1047: UserWarning: Function mol.dumps drops attribute charge because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -0.228154223327322
Calculating alchemical potentials
Doing calculation for lambda = 0.5, number of electrons = 28
Preparing input
Doing SCF calculation
converged SCF energy = -71.089986690246
Calculating alchemical potentials
Doing calculation for lambda = 1.0, number of electrons = 54
Preparing input
Doing SCF calculation
converged SCF energy = -338.557417346826
Calculating alchemical potentials
Saving results


In [ ]:
alchemical_potentials

In [ ]:
np.load('alchpots.npy')

In [ ]:
# single lambda

In [ ]:
np.where(52*lam_vals==34)

In [ ]:
0.6538461538461539*52

In [ ]:
import sys
sys.path.insert(0,'/home/misa/git_repositories/APDFT/prototyping/atomic_energies')
import alchemy_tools_pyscf as atp
from parse_pyscf import read_input
import qml
import numpy as np
import os

# get current directory
run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_001212'  #os.getcwd()
os.chdir(run_dir)
#print('Initializing')
input_parameters = read_input(os.path.join(run_dir, 'input_parameters'))
inputfile = input_parameters['structure_file']#os.path.join(basepath, com+'.xyz')
intg_meth = input_parameters['intg_meth']
basis = input_parameters['basis'] # 'def2-qzvp'
com = qml.Compound(xyz=inputfile)

lam_vals = np.arange(2, 54, 2)/52
lam_vals = np.concatenate((np.zeros(1), lam_vals))
alchemical_potentials = []
alchemical_potentials.append(np.zeros(len(com.nuclear_charges)).tolist())
mo_energies = []
mo_occs = []
dm_restart = None

#print('Starting calculations')
for lam in lam_vals[17:18]: #np.flip(lam_vals)[:-1]:
    print(lam)
    num_elec = atp.get_num_elec(lam, com.nuclear_charges.sum())
    #print('Preparing input')
    deltaZ, includeonly, mol = atp.prepare_input(com.coordinates, com.nuclear_charges, num_elec, basis)
    #print('Doing SCF calculation')
    dm, e_tot, mo_energy, mo_occ = atp.make_apdft_calc(deltaZ, dm_restart, includeonly, mol, method = "HF", **{'max_cycle':300, 'diis':True, 'init_guess': '1e'})
    #dm_restart = dm
    mo_energies.append(mo_energy)
    mo_occs.append(mo_occ)
    #print('Calculating alchemical potentials')
    alchpots_lambda = atp.calculate_alchpot(dm, includeonly, mol)
    alchemical_potentials.append(alchpots_lambda)

In [ ]:
# run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_001212'

In [ ]:
# file_raw = os.path.join(run_dir, 'alchpots_ig_atom.npy')
# alchemical_potentials_loaded = np.load(file_raw)

# # save mo energies
# file_moen = os.path.join(run_dir, 'mo_energies_ig_atom.npy')
# mo_en_loaded = np.load(file_moen)
# # save mo occupancies
# file_moocc = os.path.join(run_dir, 'mo_occupancies_ig_atom.npy')
# mo_occ_loaded = np.load(file_moocc)

In [ ]:
# alchemical_potentials_loaded[17] = alchemical_potentials[1]

In [ ]:
# mo_en_loaded[16] = mo_energies[0]

In [ ]:
# # save alchemical_potentials
# file_raw = os.path.join(run_dir, 'alchpots_ig_atom')
# np.save(file_raw, alchemical_potentials_loaded, allow_pickle=False)
# # save mo energies
# file_moen = os.path.join(run_dir, 'mo_energies_ig_atom')
# np.save(file_moen, mo_en_loaded, allow_pickle=False)